In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import zipfile
import os

# Define the zip file path (use the Google Drive path)
zip_file_path = "/content/drive/My Drive/UTKFace.zip"

# Define the extraction directory
extract_dir = "/content/UTKFace"

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check the extracted files
extracted_files = os.listdir(extract_dir)
print(f"Extracted files: {len(extracted_files)}")


In [ ]:
import os

# List all files and directories in the extracted folder
extracted_files = os.listdir(extract_dir)
print(f"Extracted files and directories: {extracted_files}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from PIL import Image
import os
from pathlib import Path

In [ ]:

# Define the directory path
path = "/content/UTKFace/UTKFace"
all_files = os.listdir(path)
print(f"Extracted files and directories: {all_files}")
# List all files and directories in the specified path
all_files = os.listdir(path)
print("All files in directory:", all_files)


In [ ]:
print(len(all_files))

In [ ]:
print(all_files[:5])

In [ ]:
np.random.seed(42)
np.random.shuffle(all_files)

In [ ]:
print(len(all_files))

In [ ]:
age_labels, gender_labels , image_paths =[],[],[]
for file in all_files:
    image_paths.append(file)
    temp = file.split("_")
    age_labels.append(temp[0])

    gender_labels.append(temp[1])


In [ ]:
image_paths

In [ ]:
data = pd.DataFrame()
data["image"] = image_paths
data["age"] = age_labels
data["gender"] = gender_labels

In [ ]:
data.head()

In [ ]:
gender_dict = {
    0 : "Male",
    1 : "Female"
}

In [ ]:
print(data.dtypes)

In [ ]:
data = data.astype({"age":"float32","gender":"int32"})

In [ ]:
print(data.dtypes)

In [ ]:
img = Image.open(path + "/" + data.image[0])
plt.imshow(img)

In [ ]:
sns.displot(data.age)

In [ ]:
# Visualize distributions
sns.histplot(data=data, x="age").set(title="Age Distribution")
sns.histplot(data=data, x="gender").set(title="Gender Distribution")

In [ ]:
from keras.preprocessing.image import load_img
files = data.iloc[0:20]
plt.figure(figsize=(15,15))

for index , file , age , gender in files.itertuples():
  plt.subplot(5,5,index+1)
  img = load_img(path + "/" + file)
  img = np.array(img)
  plt.imshow(img)
  plt.title(f"Age : {age} Gender : {gender_dict[gender]}")
  plt.axis("off")

In [ ]:
data.info()

In [ ]:
#Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Initialize the ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rescale=1./255,          # Rescale pixel values
    rotation_range=30,       # Random rotation
    width_shift_range=0.2,   # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest'      # Fill missing pixels
)

In [ ]:
sns.histplot(data = data ,x="age").set(title="Age Distribution")

In [ ]:
sns.histplot(data = data , x = "gender").set(title= "Gender Distribution")

In [ ]:
data['gender'].value_counts().plot(kind='bar')


In [ ]:
from sklearn.model_selection import train_test_split

train , test = train_test_split(data , test_size = 0.2 , random_state= 42)


In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
x_train = []
for file in train.image:
  img = load_img(path + "/" + file , color_mode = "grayscale" )
  img = img.resize((128,128), Image.Resampling.LANCZOS)
  img = np.array(img)
  x_train.append(img)


x_train = np.array(x_train)

x_train = x_train.reshape(len(x_train),128,128,1)

In [ ]:
x_train = x_train/255

In [ ]:
y_gender = np.array(train.gender)
y_age = np.array(train.age)

In [ ]:
input_size = (128,128,1)

In [ ]:
from keras.models import Model
from keras.layers import Input , Conv2D , BatchNormalization , MaxPooling2D , Flatten , Dense ,Dropout
from keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2




inputs = Input(shape= input_size)

#First Conv Block
X = Conv2D(512,(3,3),activation = "relu",kernel_initializer = glorot_uniform(seed=0))(inputs)
X = BatchNormalization(axis=3)(X)
X = MaxPooling2D((3,3))(X)
X = Dropout(0.3)(X)

#2nd Conv Block
X = Conv2D(256,(3,3),activation = "relu")(X)
X = BatchNormalization()(X)
X = MaxPooling2D((2,2),strides=(2,2))(X)
X = Dropout(0.3)(X)

#3rd Conv Block
X = Conv2D(128,(3,3),activation = "relu")(X)
X = MaxPooling2D((2,2))(X)
X = Dropout(0.3)(X)

#4th Conv Block
X = Conv2D(64,(3,3),activation = "relu")(X)
X = MaxPooling2D((2,2))(X)
X = Dropout(0.3)(X)

#Flatten and Dense Layer
X = Flatten()(X)

dense_1 = Dense(256,activation = "relu",kernel_regularizer = l2(0.01))(X)
drop_out_1 = Dropout(0.4)(dense_1)

age_dense_1 = Dense(512,activation = "relu")(X)
age_dense_2 = Dense(256, activation="relu")(age_dense_1)


output_1 = Dense(1,activation = "sigmoid" , name = "Gender_Output")(drop_out_1)
output_2 = Dense(1,activation = "linear" , name = "Age_Output")(age_dense_2)

model = Model(inputs = inputs , outputs = [output_1 , output_2])




In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanAbsoluteError, Accuracy
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

# Define your optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0005)  # Lower learning Rate for fine Tuning

model.compile(
    optimizer = optimizer,
    loss = ["binary_crossentropy","mae"],
    loss_weights = [1.0,2.0],
    metrics = [["accuracy"],[MeanAbsoluteError()]]
)

# Callbacks for early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=1,
    restore_best_weights=True
)





In [ ]:
model.summary()

In [ ]:
model_history = model.fit(x = x_train , y = [y_gender,y_age],batch_size = 32 , epochs = 20, validation_split=0.1)

In [ ]:
# Prepare the test set images
x_test = []
for file in test.image:
    img = load_img(path + "/" + file, color_mode="grayscale")
    img = img.resize((128, 128), Image.Resampling.LANCZOS)
    img = np.array(img)
    x_test.append(img)

x_test = np.array(x_test)

# Reshape the test set images to match the input shape of the model
x_test = x_test.reshape(len(x_test), 128, 128, 1)

# Normalize the test images (rescale pixel values between 0 and 1)
x_test = x_test / 255.0

# Prepare the actual test labels
y_test_gender = np.array(test.gender)
y_test_age = np.array(test.age)

# Predict gender and age on the test set
predictions = model.predict(x_test)

# Gender predictions (rounding because output is between 0 and 1)
gender_predictions = np.round(predictions[0])

# Age predictions
age_predictions = predictions[1]

# Convert gender predictions back to labels (0 for Male, 1 for Female)
gender_predicted_labels = [gender_dict[int(gender)] for gender in gender_predictions]

# Show some sample predictions
for i in range(5):  # Show the first 5 test predictions
    print(f"Image: {test.image.iloc[i]}")
    print(f"Predicted Gender: {gender_predicted_labels[i]}")
    print(f"Predicted Age: {age_predictions[i][0]:.2f}")
    print(f"Actual Gender: {gender_dict[test.gender.iloc[i]]}")
    print(f"Actual Age: {test.age.iloc[i]}")
    print("\n")


In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error , mean_squared_error

# Accuracy for gender prediction
gender_accuracy = accuracy_score(y_test_gender, gender_predictions)
print(f"Gender Prediction Accuracy: {gender_accuracy:.2f}")

# Mean Absolute Error for age prediction
age_mae = mean_absolute_error(y_test_age, age_predictions)
print(f"Age Prediction Mean Absolute Error: {age_mae:.2f}")

# Mean Squared Error for age prediction
age_mse = mean_squared_error(y_test_age, age_predictions)
print(f"Age Prediction Mean Squared Error: {age_mse:.2f}")
